In [11]:
import os
import pandas as pd
import boto3
import itables as itables
from dotenv import load_dotenv


In [12]:

# Wczytaj zmienne środowiskowe z .env
load_dotenv()

# Pobierz dane dostępowe z .env
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ENDPOINT_URL_S3 = os.getenv("AWS_ENDPOINT_URL_S3")  # np. https://fra1.digitaloceanspaces.com
BUCKET_NAME = "zadanie9"

# Tworzymy klienta S3
session = boto3.session.Session()
s3 = session.client(
	"s3",
	endpoint_url=AWS_ENDPOINT_URL_S3,
	aws_access_key_id=AWS_ACCESS_KEY_ID,
	aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Ścieżka pliku na DO
s3_file_path = "datasets/halfmarathon_clean.csv"

# Pobierz plik i zapisz go lokalnie
local_file_path = "halfmarathon_clean.csv"
s3.download_file(BUCKET_NAME, s3_file_path, local_file_path)

# Wczytaj plik do Pandas
df_clean = pd.read_csv(local_file_path)

# Sprawdzenie danych
print(df_clean.head())

   Płeć  Rocznik  5 km Czas  5 km Tempo  10 km Czas  10 km Tempo  15 km Czas  \
0     1     1992      877.0    2.923333      1755.0     2.926667      2687.0   
1     1     1986      888.0    2.960000      1783.0     2.983333      2726.0   
2     1     1996      946.0    3.153333      1883.0     3.123333      2854.0   
3     1     1988      971.0    3.236667      1930.0     3.196667      2929.0   
4     1     1995      972.0    3.240000      1955.0     3.276667      2971.0   

   15 km Tempo  20 km Czas  20 km Tempo  Tempo Stabilność    Czas     Tempo  
0     3.106667      3703.0     3.386667          0.031400  3899.0  3.080509  
1     3.143333      3788.0     3.540000          0.038000  3983.0  3.146875  
2     3.236667      3909.0     3.516667          0.024067  4104.0  3.242475  
3     3.330000      4014.0     3.616667          0.025467  4216.0  3.330963  
4     3.386667      4047.0     3.586667          0.023000  4227.0  3.339654  


In [13]:
itables.show(df_clean)

In [14]:
from pycaret.regression import setup, compare_models, models, create_model, finalize_model, save_model, load_model


In [15]:
# Zamiana spacji na podkreślniki w nazwach kolumn
df_clean.columns = df_clean.columns.str.replace(" ", "_")

# Sprawdzenie czy zmiany się zapisały
print(df_clean.columns)

Index(['Płeć', 'Rocznik', '5_km_Czas', '5_km_Tempo', '10_km_Czas',
       '10_km_Tempo', '15_km_Czas', '15_km_Tempo', '20_km_Czas', '20_km_Tempo',
       'Tempo_Stabilność', 'Czas', 'Tempo'],
      dtype='object')


In [43]:
# ✅ 1. Podział danych na modele dla kolejnych odcinków
df_10km = df_clean[["Płeć", "Rocznik", "5_km_Tempo", "10_km_Czas"]]
df_15km = df_clean[["Płeć", "Rocznik", "5_km_Tempo", "10_km_Czas", "15_km_Czas"]]
df_20km = df_clean[["Płeć", "Rocznik", "5_km_Tempo", "10_km_Czas", "15_km_Czas", "20_km_Czas"]]
df_final_time = df_clean[["Płeć", "Rocznik", "5_km_Tempo", "10_km_Czas", "15_km_Czas", "20_km_Czas", "Czas"]]

print("📌 Dane podzielone na podzbiory dla każdego etapu wyścigu.")

📌 Dane podzielone na podzbiory dla każdego etapu wyścigu.


In [45]:
# 📌 2. Ustawienie PyCaret dla każdego modelu

from pycaret.regression import setup, create_model, finalize_model, save_model

# ✅ Model dla czasu na 10 km
setup_10km = setup(data=df_10km, target="10_km_Czas", verbose=False)
model_10km = create_model('lr')  # Regresja liniowa
final_model_10km = finalize_model(model_10km)
save_model(final_model_10km, "model_10km")

# ✅ Model dla czasu na 15 km
setup_15km = setup(data=df_15km, target="15_km_Czas", verbose=False)
model_15km = create_model('lr')  
final_model_15km = finalize_model(model_15km)
save_model(final_model_15km, "model_15km")

# ✅ Model dla czasu na 20 km
setup_20km = setup(data=df_20km, target="20_km_Czas", verbose=False)
model_20km = create_model('lr')  
final_model_20km = finalize_model(model_20km)
save_model(final_model_20km, "model_20km")

# ✅ Model dla całkowitego czasu półmaratonu
setup_final = setup(data=df_final_time, target="Czas", verbose=False)
model_final_time = create_model('lr')  
final_model_final_time = finalize_model(model_final_time)
save_model(final_model_final_time, "model_final_time")

print("✅ Wszystkie modele zostały wytrenowane i zapisane!")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,56.0528,13712.3090,117.0996,0.9459,0.1096,0.0161
1,52.0974,5702.7780,75.5167,0.9777,0.0212,0.0151
2,50.7888,6006.2982,77.5003,0.9766,0.0211,0.0146
3,52.2600,5980.9193,77.3364,0.9775,0.0219,0.0151
4,53.7624,8034.9917,89.6381,0.9680,0.0239,0.0154
5,54.0895,8401.7568,91.6611,0.9693,0.0221,0.0153
6,55.4051,6341.0060,79.6304,0.9769,0.0223,0.0159
7,51.1609,5742.7560,75.7810,0.9786,0.0208,0.0148
8,48.6388,4583.2995,67.7001,0.9830,0.0189,0.0141


Transformation Pipeline and Model Successfully Saved


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,51.0917,6935.5588,83.2800,0.9889,0.0149,0.0096
1,51.6176,7638.4425,87.3982,0.9886,0.0163,0.0098
2,48.9195,5403.8595,73.5109,0.9915,0.0131,0.0092
3,50.8456,5521.3051,74.3055,0.9914,0.0136,0.0096
4,50.3125,7473.8902,86.4517,0.9878,0.0162,0.0094
5,49.2091,5580.7010,74.7041,0.9911,0.0135,0.0093
6,52.1628,7157.7325,84.6034,0.9887,0.0152,0.0098
7,50.4586,6976.2572,83.5240,0.9891,0.0140,0.0093
8,50.1467,6971.3346,83.4945,0.9896,0.0148,0.0094


Transformation Pipeline and Model Successfully Saved


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,84.4661,15491.4906,124.4648,0.9884,0.0162,0.0116
1,81.3205,13599.1198,116.6153,0.9893,0.0153,0.0112
2,82.2776,14727.4016,121.3565,0.9884,0.0163,0.0115
3,81.6958,19761.8524,140.5769,0.9834,0.0187,0.0113
4,84.4127,18013.2044,134.2133,0.9870,0.0181,0.0116
5,84.9763,18275.1042,135.1854,0.9862,0.0177,0.0117
6,82.8159,14397.6622,119.9903,0.9885,0.0159,0.0114
7,82.3128,13079.1268,114.3640,0.9898,0.0152,0.0114
8,80.4403,13252.3853,115.1190,0.9902,0.0151,0.0111


Transformation Pipeline and Model Successfully Saved


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,35.6201,2092.1069,45.7396,0.9985,0.0059,0.0048
1,36.3835,4571.5870,67.6135,0.9967,0.0084,0.0049
2,36.3376,2081.9809,45.6287,0.9985,0.0060,0.0049
3,37.9515,3571.6594,59.7634,0.9976,0.0073,0.0051
4,36.9619,2533.0451,50.3294,0.9982,0.0063,0.0049
5,35.8700,2095.3418,45.7749,0.9987,0.0059,0.0048
6,41.5179,24979.9844,158.0506,0.9837,0.0190,0.0054
7,40.6960,11861.8299,108.9120,0.9915,0.0131,0.0054
8,36.0727,2127.1308,46.1208,0.9985,0.0060,0.0048


Transformation Pipeline and Model Successfully Saved
✅ Wszystkie modele zostały wytrenowane i zapisane!
